In [1]:
# Will reload modeules after this when they change!
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import json
import os
from string import digits
import glob
import re

# RDKit
import rdkit
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Draw

from molmass import Formula

from main_functions import *

# Molecule database merge with experimental MS/MS data for METASPACE.

This script merges a molecule databae with experimental MS/MS data.  These data can then be submitted to METASPACE to search for nuetral loss and in-source fragments within datasets.

See: /Users/dis/PycharmProjects/word2vec/main_functions.py

In [3]:
# Load reference database by path:
path = '/Users/dis/PycharmProjects/core_metabolome/core_metabolome_v3.pickle'
ref_db = load_molecule_database(path)

In [4]:
# Parses GNPS_json to GNPS_df:
GNPS_json = '/Users/dis/PycharmProjects/word2vec/ALL_GNPS.json'
GNPS_df = parse_GNPS_json(GNPS_json)

In [5]:
# Needs structures to rescue 'PNNL-LIPIDS-POSITIVE'
# Drop for now by turning code line 173 off.
GNPS_df.library_membership.value_counts()

MONA                                               30784
MASSBANK                                            8729
GNPS-NIH-NATURALPRODUCTSLIBRARY_ROUND2_POSITIVE     5725
GNPS-LIBRARY                                        2662
GNPS-NIH-SMALLMOLECULEPHARMACOLOGICALLYACTIVE       1398
MASSBANKEU                                          1287
GNPS-NIH-NATURALPRODUCTSLIBRARY                     1266
GNPS-NIH-NATURALPRODUCTSLIBRARY_ROUND2_NEGATIVE     1041
RESPECT                                              981
SUMNER                                               795
GNPS-MSMLS                                           787
BILELIB19                                            747
GNPS-COLLECTIONS-PESTICIDES-POSITIVE                 644
GNPS-SELLECKCHEM-FDA-PART2                           592
GNPS-EMBL-MCF                                        585
CASMI                                                562
LDB_POSITIVE                                         461
GNPS-NIH-CLINICALCOLLECTION1   

In [6]:
# Identifies subset of GNPS with experimental specrta in ref db:
# Result: # 3920 spectra for 508 can_no_stereo_smiles
# Issue index 31383 when PNNL is included
GNPS_hits_df = search_GNPS_targets(ref_db, GNPS_df)

In [7]:
GNPS_hits_df.library_membership.value_counts()

MONA                                               1553
MASSBANK                                            794
GNPS-MSMLS                                          434
RESPECT                                             303
GNPS-EMBL-MCF                                       259
BILELIB19                                           191
GNPS-LIBRARY                                        111
SUMNER                                               95
GNPS-NIH-SMALLMOLECULEPHARMACOLOGICALLYACTIVE        41
GNPS-NIH-NATURALPRODUCTSLIBRARY_ROUND2_POSITIVE      25
GNPS-NIST14-MATCHES                                  19
GNPS-COLLECTIONS-MISC                                18
MASSBANKEU                                           18
CASMI                                                14
GNPS-NIH-CLINICALCOLLECTION1                         10
GNPS-SELLECKCHEM-FDA-PART2                           10
LDB_NEGATIVE                                          7
GNPS-NIH-NATURALPRODUCTSLIBRARY                 

In [8]:
GNPS_hits_df.Adduct.value_counts()

M+H        1982
M-H         923
M+Na        464
M+K         219
M+NH4+      176
M-H2O+H     135
M+NH4        13
M+            6
M-H2O-H       1
M+Cl          1
Name: Adduct, dtype: int64

In [9]:
# Identifies subset of Mona with experimental specrta in ref db, runs in a few hours
# Result: # 7646 spectra for 648 inchi matches
# Mona_hits_df = search_MONA(ref_db)

# Reload existing:
Mona_hits_df = pd.read_pickle('mona_2020_Apr_15.pickle')

# Filters Mona_hits_df for high-res instruments, good adducts, and <20 ppm error.
# Result: # 3437 spectra for 535 inchi matches
Mona_hits_df = parse_MONA_out(Mona_hits_df)

In [10]:
Mona_hits_df.adduct.value_counts()

M+H        2805
M+Na        254
M+K         153
M+NH4+      142
M-H2O+H      56
M+           27
Name: adduct, dtype: int64

In [11]:
Mona_hits_df.source.value_counts()

MassBank                                   1197
Vaniya/Fiehn Natural Products Library       836
unknown                                     471
Fiehn HILIC Library                         379
GNPS                                        296
ReSpect                                     199
RIKEN PlaSMA Authentic Standard Library      59
Name: source, dtype: int64

# Joinging HMDB predicted
3. Use to filter reference db on id
4. Repeat for theoretical of no expirimental

4. Copy files .txt to correct folder based on name

n. flag if hit found somehow? do not in of join...

In [17]:
rerun = False

#Import experimental and computational HMDB spectra, slow.
if rerun == True:
    hmdb_pos_exptl_df = parse_hmdb('/Users/dis/Desktop/hmdb_LCMS/hmdb_experimental_msms_spectra',
                               theo_flag=False)
    hmdb_pos_theo_df = parse_hmdb('/Users/dis/Desktop/hmdb_LCMS/hmdb_predicted_msms_spectra', 
                              theo_flag=True)
    
    hmdb_pos_exptl_df.to_pickle('hmdb_pos_exptl_df.pickle')
    hmdb_pos_theo_df.to_pickle('hmdb_pos_theo_df.pickle')

# Reload without reprocessing
else:
    hmdb_pos_exptl_df = pd.read_pickle('hmdb_pos_exptl_df.pickle')
    hmdb_pos_theo_df = pd.read_pickle('hmdb_pos_theo_df.pickle')

In [22]:
# Cleans up ref db for only entries with experimental MS/MS spectra
# Without theoretical specta: 855 id's, with: 11440
df = preparser_Sirius(ref_db,
                      GNPS_hits_df,
                      Mona_hits_df,
                      hmdb_pos_exptl_df,
                      hmdb_pos_theo_df)

## To do Tuesday:
## 1. Call Theo 8 am
## 2. Write function and test with small subset lines 502-515.

In [ ]:
# Runs main Sirius loop
sirius_output_df = loop_Sirius(df, 
                               Mona_hits_df, 
                               GNPS_hits_df,
                               hmdb_pos_exptl_df,
                               hmdb_pos_theo_df)

In [ ]:
## Test above fxn

In [ ]:
# Reload without rerunning
# sirius_output_df.to_pickle('sirius_output_df.pickle')
sirius_output_df = pd.read_pickle('sirius_output_df.pickle')

In [ ]:
# Clean-up and merge output with MS2 spectra
clean_output_df = results_clean_up(df, sirius_output_df)

# Clean-up database for METASPACE
METASPACE_db = output_METASPACE(clean_output_df, out_name)

# Export entire db
METASPACE_db.iloc[:,0:4].to_csv('whole_body_msms_test_v2.csv', sep='\t')

In [ ]:
## Make loop filtering all 10x datasets below.
## Run: 1) entire DB, 2) consensus 10x, 3) each

# Filter DB for observed MS1 and export:
f2_ids = list(pd.read_csv('f2.txt').moleculeIds)
METASPACE_db = METASPACE_db[METASPACE_db.old_id.isin(f2_ids)]
METASPACE_db.iloc[:,0:4].to_csv('whole_body_msms_test_v3.csv', sep='\t')

In [ ]:
## Report this to file
# Before filtering 10019 fragments for 776 parents
# After fitering 1228 fragments for 111 parents 

In [ ]:
## Make a loop here too, but later...
# Load search results from METASPACE run off line after database
ms_out = pd.read_csv('metaspace_annotations.csv', header=2)
ms_out = extract_results_METASPACE(ms_out)
ms_out.to_pickle('ms_out_v3_test01.pickle')

Local plotting here:
http://localhost:8888/notebooks/PycharmProjects/word2vec/IMS_plotter_v2.ipynb

# Friday:
1. Key questions:
    1.1. How often it works? 
    1.2. How much overlap in m/z is between ions? 
    1.3. How much overlap in space?
2. API access ion images and put in folders as pairs
3. Add libraries HMDB, all Mona dl, PNNL in GNPS

Next Jupyter notebook extract data from METASPACE:
http://localhost:8888/notebooks/PycharmProjects/word2vec/export_ion_mages.ipynb
Next Jupyter notebook extract data locally:
http://localhost:8888/notebooks/PycharmProjects/word2vec/IMS_plotter_v2.ipynb